<a href="https://colab.research.google.com/github/jaya-deepthi/deepthiadsexperiencial/blob/main/adsexperiencial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

matches=pd.read_csv("/content/match_data.csv")

# Data Cleaning

In [ ]:
conditions = [matches["venue"] == "Rajiv Gandhi International Stadium, Uppal",matches["venue"] == "Maharashtra Cricket Association Stadium",
              matches["venue"] == "Saurashtra Cricket Association Stadium", matches["venue"] == "Holkar Cricket Stadium",
              matches["venue"] == "M Chinnaswamy Stadium",matches["venue"] == "Wankhede Stadium",
              matches["venue"] == "Eden Gardens",matches["venue"] == "Feroz Shah Kotla",
              matches["venue"] == "Punjab Cricket Association IS Bindra Stadium, Mohali",matches["venue"] == "Green Park",
              matches["venue"] == "Punjab Cricket Association Stadium, Mohali",matches["venue"] == "Dr DY Patil Sports Academy",
              matches["venue"] == "Sawai Mansingh Stadium", matches["venue"] == "MA Chidambaram Stadium, Chepauk", 
              matches["venue"] == "Newlands", matches["venue"] == "St George's Park" , 
              matches["venue"] == "Kingsmead", matches["venue"] == "SuperSport Park",
              matches["venue"] == "Buffalo Park", matches["venue"] == "New Wanderers Stadium",
              matches["venue"] == "De Beers Diamond Oval", matches["venue"] == "OUTsurance Oval", 
              matches["venue"] == "Brabourne Stadium",matches["venue"] == "Sardar Patel Stadium", 
              matches["venue"] == "Barabati Stadium", matches["venue"] == "Vidarbha Cricket Association Stadium, Jamtha",
              matches["venue"] == "Himachal Pradesh Cricket Association Stadium",matches["venue"] == "Nehru Stadium",
              matches["venue"] == "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium",matches["venue"] == "Subrata Roy Sahara Stadium",
              matches["venue"] == "Shaheed Veer Narayan Singh International Stadium",matches["venue"] == "JSCA International Stadium Complex",
              matches["venue"] == "Sheikh Zayed Stadium",matches["venue"] == "Sharjah Cricket Stadium",
              matches["venue"] == "Dubai International Cricket Stadium",matches["venue"] == "M. A. Chidambaram Stadium",
              matches["venue"] == "Feroz Shah Kotla Ground",matches["venue"] == "M. Chinnaswamy Stadium",
              matches["venue"] == "Rajiv Gandhi Intl. Cricket Stadium" ,matches["venue"] == "IS Bindra Stadium",matches["venue"] == "ACA-VDCA Stadium"]
values = ['Hyderabad', 'Mumbai', 'Rajkot',"Indore","Bengaluru","Mumbai","Kolkata","Delhi","Mohali","Kanpur","Mohali","Pune","Jaipur","Chennai","Cape Town","Port Elizabeth","Durban",
          "Centurion",'Eastern Cape','Johannesburg','Northern Cape','Bloemfontein','Mumbai','Ahmedabad','Cuttack','Jamtha','Dharamshala','Chennai','Visakhapatnam','Pune','Raipur','Ranchi',
          'Abu Dhabi','Sharjah','Dubai','Chennai','Delhi','Bengaluru','Hyderabad','Mohali','Visakhapatnam']
matches['city'] = np.where(matches['city'].isnull(),
                              np.select(conditions, values),
                              matches['city'])
#removing the rows containing null values
matches=matches[matches["winner"].notna()]

In [ ]:
matches["team2"]=matches["team2"].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches["team1"]=matches["team1"].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches["winner"]=matches["winner"].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches["toss_winner"]=matches["toss_winner"].replace("Rising Pune Supergiant","Rising Pune Supergiants")

# Feature Engineering

here I am using label encoder to encode the labels into numeric form

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
matches["team1"]=encoder.fit_transform(matches["team1"])
matches["team2"]=encoder.fit_transform(matches["team2"])
matches["winner"]=encoder.fit_transform(matches["winner"].astype(str))
matches["toss_winner"]=encoder.fit_transform(matches["toss_winner"])
matches["venue"]=encoder.fit_transform(matches["venue"])

The value in the column team1_win will be 1 if team1 wins else 0, for column team1_toss_decision it will be 1 if team1 wins the toss and finally team1_bat will be 1 if team1 is batting first.

In [ ]:
matches.loc[matches["winner"]==matches["team1"],"team1_win"]=1
matches.loc[matches["winner"]!=matches["team1"],"team1_win"]=0

#outcome variable team1_toss_win as a value of team1 winning the toss
matches.loc[matches["toss_winner"]==matches["team1"],"team1_toss_win"]=1
matches.loc[matches["toss_winner"]!=matches["team1"],"team1_toss_win"]=0

#outcome variable team1_bat to depict if team1 bats first
matches["team1_bat"]=0
matches.loc[(matches["team1_toss_win"]==1) & (matches["toss_decision"]=="bat"),"team1_bat"]=1


# Feature Selection

Here I am removing the coloums having nearly 1 correlation coe with the other coloums.

In [ ]:
prediction_df=matches[["team1","team2","team1_toss_win","team1_bat","team1_win","venue"]]

#finding the higly correlated features
correlated_features = set()
correlation_matrix = prediction_df.drop('team1_win', axis=1).corr()
print(correlation_matrix)

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            column = correlation_matrix.columns[i]
            correlated_features.add(column)
            
prediction_df.drop(columns=correlated_features)

                   team1     team2  team1_toss_win  team1_bat     venue
team1           1.000000 -0.113733       -0.056944  -0.142246  0.172634
team2          -0.113733  1.000000        0.046833  -0.003217  0.006020
team1_toss_win -0.056944  0.046833        1.000000   0.521315  0.001372
team1_bat      -0.142246 -0.003217        0.521315   1.000000 -0.011217
venue           0.172634  0.006020        0.001372  -0.011217  1.000000


,team1,team2,team1_toss_win,team1_bat,team1_win,venue
0,12,1,0.0,0,1.0,4
1,3,7,1.0,1,1.0,8
2,8,7,1.0,0,1.0,35
3,11,12,1.0,0,0.0,17
4,0,1,1.0,1,1.0,4
...,...,...,...,...,...,...
741,13,8,1.0,0,1.0,24
742,0,1,1.0,1,1.0,2
743,12,8,0.0,0,0.0,14
744,5,12,1.0,0,1.0,6


In [ ]:
X = prediction_df.drop('team1_win', axis=1)
target = prediction_df['team1_win']
target=target.astype(int)
import sklearn.linear_model as sk
from sklearn.feature_selection import RFE
logreg = sk.LogisticRegressionCV()
rfe = RFE(logreg, 20)
rfe = rfe.fit(X, target.values.ravel())
#Checking for the features of they are important
print(rfe.support_)

[ True  True  True  True  True]


# Splitting data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=0,shuffle=True)


# Build the models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of Logistic Regression Classifier on test set: {:.4f}'.format(logreg.score(X_test, y_test)))

#Decision Tree Classifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred = dtree.predict(X_test)
print('Accuracy of Decision Tree Classifier on test set: {:.4f}'.format(dtree.score(X_test, y_test)))
from sklearn.svm import SVC
#SVM
svm=SVC()
svm.fit(X_train,y_train)
y_pred = svm.predict(X_test)
print('Accuracy of SVM Classifier on test set: {:.4f}'.format(svm.score(X_test, y_test)))
from sklearn.ensemble import RandomForestClassifier
#Random Forest Classifier
randomForest= RandomForestClassifier(n_estimators=100)
randomForest.fit(X_train,y_train)
y_pred = randomForest.predict(X_test)
print('Accuracy of Random Forest Classifier on test set: {:.4f}'.format(randomForest.score(X_test, y_test)))


Accuracy of Logistic Regression Classifier on test set: 0.5578
Accuracy of Decision Tree Classifier on test set: 0.5714
Accuracy of SVM Classifier on test set: 0.5578
Accuracy of Random Forest Classifier on test set: 0.5510


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Confusion Matrix for random forest:\n")
print(confusion_matrix(y_pred,y_test))

Confusion Matrix for random forest:

[[29 30]
 [36 52]]


# Models significance testing

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv
t, p = paired_ttest_5x2cv(estimator1=logreg, estimator2=randomForest, X=X_train, y=y_train, scoring='accuracy', random_seed=1)

In [ ]:
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))

P-value: 0.750, t-Statistic: -0.337


In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv
t1, p1 = paired_ttest_5x2cv(estimator1=logreg, estimator2=dtree, X=X_train, y=y_train, scoring='accuracy', random_seed=1)

In [ ]:
print('P-value: %.3f, t-Statistic: %.3f' % (p1, t1))

P-value: 0.316, t-Statistic: 1.114


In [ ]:
if p <= 0.05:
	print('The Null Hypothesis Rejected : Difference between mean performance is probably real')
else:
	print('Fail to Reject the Null Hypothesis : Algorithms probably have the same performance')

Fail to Reject the Null Hypothesis : Algorithms probably have the same performance


In [ ]:
if p1 <= 0.05:
	print('The Null Hypothesis Rejected : Difference between mean performance is probably real')
else:
	print('Fail to Reject the Null Hypothesis : Algorithms probably have the same performance')

Fail to Reject the Null Hypothesis : Algorithms probably have the same performance


Here the accuracy scores are almost same. But here the accuracy scores are less because of the data set. Data set i chose has many null values & is not so perfect. which we can see thorugh EDA performed last time. Even though the accuracy is not high enough to be useful, it gives a basic idea about the strategies and methodologies used in designing a solution to the Machine Learning problem.